In [ ]:
# load the LSST stack
# source ~/lsst_devel/LSST/lsstsw/bin/setup.sh 
# setup pipe_tasks -t b1843
# setup -j -r ~/lsst_devel/LSST/repos/ticket3704/obs_decam/

In [ ]:
import lsst.pipe.base as pipeBase
from lsst.pipe.tasks.processCcd import ProcessCcdTask
from lsst.obs.decam.decamNullIsr import DecamNullIsrTask

In [ ]:
from lsst.ip.isr import IsrTask
import lsst.pex.config as pexConfig
import lsst.pipe.base as pipeBase
from lsst.pipe.tasks.calibrate import CalibrateTask, CalibrateConfig
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig 

In [ ]:
# try out one exposure
visit=289851
ccdnum = 22
filterName='g'

In [ ]:
import lsst.daf.persistence as dafPersist
butler = dafPersist.Butler('/Users/yusra/calexp_dir_Blind14A_10')

In [ ]:
# read in exposure
# ccd level keys are visit, ccdnum and filter. filter for this dataset is 'g'
# IF you want them to look nice: use github.com:yalsayyad/obs_decam (look out for branch number that reads in 
#        crblasted files instead of raw files)

exposure = butler.get("instcal", visit=visit, ccdnum=ccdnum, filter='g', immediate=True)
exposureIdInfo = butler.get("expIdInfo", visit=visit, ccdnum=ccdnum, filter='g', immediate=True)
exposure.writeFits('filename.fits')


In [ ]:
# Characterize Image
charImageConfig = CharacterizeImageConfig()
charImage = CharacterizeImageTask()
# charImage.characterize(exposure, exposureIdInfo=exposureIdInfo,  background=None) 
# OR
charRes = charImage.characterize(exposure) #Everything else CAN be None
exposure = charRes.exposure
bkgd = charRes.background

In [ ]:
# Calibrate Image
calibrateConfig = CalibrateConfig(doPhotoCal=False, doAstrometry=False)
calibrateTask = CalibrateTask(config=calibrateConfig)
calibRes = calibrateTask.calibrate(exposure, 
                        exposureIdInfo=None, background=bkgd, icSourceCat=None)

# In original https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/processCcd.py
# the run method is called like:             calibRes = self.calibrate.run(
#                dataRef = sensorRef,
#                exposure = charRes.exposure,
#                background = charRes.background,
#                doUnpersist = False,
#                icSourceCat = charRes.sourceCat,
#            )



In [ ]:
# Set results from processCCD, these are going to be the inputs to makeCoaddTempExp

# charRes = charRes
# calibRes = calibRes if self.config.doCalibrate else None
# exposure = exposure
# background = calibRes.background if self.config.doCalibrate else charRes.background


In [ ]:
# Next step is done per patch:



In [ ]:
# to find out which patch overlaps the one ccd we processed ("exposure" now contains the "calexp")
wcs = exposure.getWcs()

tract = hits_skymap[0]

tract.findPatch(wcs.pixelToSky(1000,1000))

# answer: PatchInfo(index=(16, 17), innerBBox=Box2I((32000, 34000), (33999, 35999)), outerBBox=Box2I((31900, 33900), (34099, 36099)))

In [ ]:
def getSkyInfo(skyMap, xIndex, yIndex, tractId=0):
    tractInfo = skyMap[tractId]
    # patch format is "xIndex,yIndex"
    patchInfo = tractInfo.getPatchInfo((xIndex, yIndex))
    return pipeBase.Struct(
        skyMap = skyMap,
        tractInfo = tractInfo,
        patchInfo = patchInfo,
        wcs = tractInfo.getWcs(),
        bbox = patchInfo.getOuterBBox(),
    )


skyInfo = getSkyInfo(hits_skymap, 16, 17, tractId=0)





In [ ]:
# For each patch: 
#     For each visit that overlaps the patch (assume all):
#        makeCoaddTempExp for every visit by doing...
#        For each ccd in that visit that overlaps the patch:
#            warpAndPsfMatch.run
#            coaddUtils.copyGoodPixels  

In [ ]:
# for this part we are going to assume that we are deep in the loop:
xIndex, yIndex = (16, 17)
filterName='g'
visit=289851
ccdnum = 22

In [ ]:
from lsst.pipe.tasks.makeCoaddTempExp import MakeCoaddTempExpTask, MakeCoaddTempExpConfig
import lsst.afw.image as afwImage
import lsst.coadd.utils as coaddUtils
from lsst.meas.algorithms import CoaddPsf
import numpy
makeCTEConfig = MakeCoaddTempExpConfig()
makeCTE = MakeCoaddTempExpTask(config=makeCTEConfig)




In [ ]:
coaddTempExp = afwImage.ExposureF(skyInfo.bbox, skyInfo.wcs)
coaddTempExp.getMaskedImage().set(numpy.nan, afwImage.MaskU.getPlaneBitMask("NO_DATA"), numpy.inf)
totGoodPix = 0
didSetMetadata = False
modelPsf = makeCTEConfig.modelPsf.apply() if makeCTEConfig.doPsfMatch else None
# number of 
ccdsInVisit = 1
inputRecorder =  makeCTE.inputRecorder.makeCoaddTempExpRecorder(visit, ccdsInVisit)
ccdId = exposure.getId()

### NOW you start the loop over the 1-4 ccd's IN THIS VISIT that overlap this patch
numGoodPix = 0
calExp = exposure
# We MIGHT need to subtract the backgrounds here. Double check.
warpedCcdExp = makeCTE.warpAndPsfMatch.run(calExp, modelPsf=modelPsf, 
                                           wcs=skyInfo.wcs,maxBBox=skyInfo.bbox).exposure
if didSetMetadata:
    mimg = exposure.getMaskedImage()
    mimg *= (coaddTempExp.getCalib().getFluxMag0()[0] / exposure.getCalib().getFluxMag0()[0])
    del mimg

numGoodPix = coaddUtils.copyGoodPixels(coaddTempExp.getMaskedImage(),
                                       warpedCcdExp.getMaskedImage(),
                                       makeCTE.getBadPixelMask())
totGoodPix += numGoodPix
if numGoodPix > 0 and not didSetMetadata:
    coaddTempExp.setCalib(warpedCcdExp.getCalib())
    coaddTempExp.setFilter(warpedCcdExp.getFilter())
    didSetMetadata = True

inputRecorder.addCalExp(calExp, ccdId, numGoodPix)

##### End loop over ccds here:
inputRecorder.finish(coaddTempExp, totGoodPix)
if totGoodPix > 0 and didSetMetadata:
coaddTempExp.setPsf(modelPsf if makeCTEConfig.doPsfMatch else
                    CoaddPsf(inputRecorder.coaddInputs.ccds, skyInfo.wcs))
    